In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
df = pd.read_csv("US Mass shootings - Sheet1.csv")

In [3]:
cities_us = pd.read_csv("US_cities.csv")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
cities_us.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude
2907718,us,abanda,Abanda,AL,NaN,33.100833,-85.529722
2907719,us,abbeville,Abbeville,AL,NaN,31.571667,-85.250556
2907720,us,abbot springs,Abbot Springs,AL,NaN,33.360833,-86.481667
2907721,us,abel,Abel,AL,NaN,33.548611,-85.712500
2907722,us,abercrombie,Abercrombie,AL,NaN,32.848611,-87.165000


In [6]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [7]:
us_state_abbrev=pd.Series(us_state_abbrev, name='Code')

In [8]:
us_state_abbrev = us_state_abbrev.reset_index()

In [9]:
us_state_abbrev.head()

,index,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [10]:
df_merged =df.merge(us_state_abbrev, left_on="State", right_on="index", how="left")

In [11]:
df_merged.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,Year,index,Code
0,31-Dec-13,New York,Brooklyn,60 Glenmore Ave,0,6,2013,New York,NY
1,28-Dec-13,Alabama,Montgomery,954 Highland Ave,3,5,2013,Alabama,AL
2,26-Dec-13,Louisiana,Slidell,2144 First St,2,6,2013,Louisiana,LA
3,26-Dec-13,Louisiana,Lockport,313 Tenth St,3,3,2013,Louisiana,LA
4,25-Dec-13,New Jersey,Irvington,Nye Avenue and 21st Street,3,2,2013,New Jersey,NJ


In [12]:
df_merged.drop("index",1,inplace=True)


In [13]:
df_merged.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,Year,Code
0,31-Dec-13,New York,Brooklyn,60 Glenmore Ave,0,6,2013,NY
1,28-Dec-13,Alabama,Montgomery,954 Highland Ave,3,5,2013,AL
2,26-Dec-13,Louisiana,Slidell,2144 First St,2,6,2013,LA
3,26-Dec-13,Louisiana,Lockport,313 Tenth St,3,3,2013,LA
4,25-Dec-13,New Jersey,Irvington,Nye Avenue and 21st Street,3,2,2013,NJ


In [14]:
df_merged["City Or County"] = df_merged["City Or County"].str.replace("\(.*\)", "")

In [15]:
df_merged["City Or County"] = df_merged["City Or County"].str.strip()

In [16]:
df_merged_lon_lat = df_merged.merge(cities_us, left_on=["City Or County", "Code"], right_on=["AccentCity","Region"], how="left")

In [17]:
df_merged_lon_lat.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,Year,Code,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,31-Dec-13,New York,Brooklyn,60 Glenmore Ave,0,6,2013,NY,us,brooklyn,Brooklyn,NY,NaN,40.650000,-73.950000
1,28-Dec-13,Alabama,Montgomery,954 Highland Ave,3,5,2013,AL,us,montgomery,Montgomery,AL,198325.0,32.366667,-86.300000
2,26-Dec-13,Louisiana,Slidell,2144 First St,2,6,2013,LA,us,slidell,Slidell,LA,27799.0,30.275000,-89.781111
3,26-Dec-13,Louisiana,Lockport,313 Tenth St,3,3,2013,LA,us,lockport,Lockport,LA,NaN,29.645833,-90.539167
4,25-Dec-13,New Jersey,Irvington,Nye Avenue and 21st Street,3,2,2013,NJ,us,irvington,Irvington,NJ,61323.0,40.732222,-74.235278


In [18]:
df_merged_lon_lat["test"] = df_merged_lon_lat["Latitude"].isnull()

In [19]:
df_merged_lon_lat[df_merged_lon_lat["test"] == True]

,Incident Date,State,City Or County,Address,# Killed,# Injured,Year,Code,Country,City,AccentCity,Region,Population,Latitude,Longitude,test
32,05-Nov-13,District of Columbia,Washington,3900 block of Minnesota Ave,0,4,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
70,16-Sep-13,District of Columbia,Washington Navy Yard,1336 Isaac Hull Ave,11,3,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
75,12-Sep-13,District of Columbia,Washington,First Street and Pierce Street NW,1,3,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
121,13-Jul-13,District of Columbia,Washington,2700 block of Stanton Road,0,4,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
164,02-Jun-13,Georgia,Lagrange,Union Street,0,7,2013,GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
221,11-Mar-13,District of Columbia,Washington,1200 North Capitol Street,0,13,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
239,03-Feb-13,California,Yuba,5800 block of Poplar Avenue,1,3,2013,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
242,26-Jan-13,District of Columbia,Washington,2403 Benning Road Northeast,0,5,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
252,01-Jan-13,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,2013,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
257,26-Dec-14,Illinois,East St. Louis,2500 block of Summit Avenue,1,3,2014,IL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [20]:
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Washington", 'Latitude'] = "47.751076"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Washington", 'Longitude'] = "-120.740135"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Sandy", 'Latitude'] = "40.571563"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Sandy", 'Longitude'] = "-111.852286"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Dekalb", 'Latitude'] = "33.7956"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Dekalb", 'Longitude'] = "-84.2279"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Dekalb County", 'Latitude'] = "33.7956"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Dekalb County", 'Longitude'] = "-84.2279"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Lagrange", 'Latitude'] = "33.037880"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Lagrange", 'Longitude'] = "-85.030060"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Yuba", 'Latitude'] = "39.136986"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Yuba", 'Longitude'] = "-121.607841"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Mckeesport", 'Latitude'] = "40.3470671"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Mckeesport", 'Longitude'] = "-79.8641232"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "East St. Louis", 'Latitude'] = "38.624514"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "East St. Louis", 'Longitude'] = "-90.1506465"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Accomack County", 'Latitude'] = "37.7501289"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Accomack County", 'Longitude'] = "-75.6663182"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Queens", 'Latitude'] = "40.742054"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Queens", 'Longitude'] = "-73.769417"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Darlington County", 'Latitude'] = "34.3650"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Darlington County", 'Longitude'] = "-80.0088"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Panola County", 'Latitude'] = "32.1920"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Panola County", 'Longitude'] = "-94.3154"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Bartow County", 'Latitude'] = "34.1659305"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Bartow County", 'Longitude'] = "-84.797716"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Manhattan", 'Latitude'] = "40.758896"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Manhattan", 'Longitude'] = "-73.985130"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Lookout Valley", 'Latitude'] = "34.9939646"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Lookout Valley", 'Longitude'] = "-85.3746811"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Fort Hood", 'Latitude'] = "31.2548307"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Fort Hood", 'Longitude'] = "-97.709127"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Orange County", 'Latitude'] = "33.7174708"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Orange County", 'Longitude'] = "-117.83114280000001"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Miami-dade", 'Latitude'] = "25.778135"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Miami-dade", 'Longitude'] = "-80.179100"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Lovejoy", 'Latitude'] = "40.5369809"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Lovejoy", 'Longitude'] = "-87.7039152"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Grulla", 'Latitude'] = "26.285343"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Grulla", 'Longitude'] = "-98.644474"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Opa Locka", 'Latitude'] = "25.9023168"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Opa Locka", 'Longitude'] = "-80.25032709999999"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Pembroke Township", 'Latitude'] = "45.81681"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Pembroke Township", 'Longitude'] = "-77.11616"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Winston Salem", 'Latitude'] = "36.099861"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Winston Salem", 'Longitude'] = "-80.244217"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Hot Springs National Park", 'Latitude'] = "34.5313219"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Hot Springs National Park", 'Longitude'] = "-93.0637411"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Washington Navy Yard", 'Latitude'] = "38.8746334"
df_merged_lon_lat.loc[df_merged_lon_lat["City Or County"] == "Washington Navy Yard", 'Longitude'] = "-76.9945473"




In [21]:
df_merged_lon_lat.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,Year,Code,Country,City,AccentCity,Region,Population,Latitude,Longitude,test
0,31-Dec-13,New York,Brooklyn,60 Glenmore Ave,0,6,2013,NY,us,brooklyn,Brooklyn,NY,NaN,40.65,-73.95,False
1,28-Dec-13,Alabama,Montgomery,954 Highland Ave,3,5,2013,AL,us,montgomery,Montgomery,AL,198325.0,32.3667,-86.3,False
2,26-Dec-13,Louisiana,Slidell,2144 First St,2,6,2013,LA,us,slidell,Slidell,LA,27799.0,30.275,-89.7811,False
3,26-Dec-13,Louisiana,Lockport,313 Tenth St,3,3,2013,LA,us,lockport,Lockport,LA,NaN,29.6458,-90.5392,False
4,25-Dec-13,New Jersey,Irvington,Nye Avenue and 21st Street,3,2,2013,NJ,us,irvington,Irvington,NJ,61323.0,40.7322,-74.2353,False


In [22]:
df_merged_lon_lat.drop(["Region", "Population", "City", "AccentCity", "Region"],1, inplace = True)

In [23]:
df_merged_lon_lat.columns = ["Date", "State", "City", "Address", "Killed", "Injured", "Year", "Code", "Country", "Latitude", "Longitude", "test"]

In [24]:
df_merged_lon_lat["Latitude"] = df_merged_lon_lat["Latitude"].astype(float)
df_merged_lon_lat["Longitude"] = df_merged_lon_lat["Longitude"].astype(float)

In [31]:
df_merged_lon_lat.sort_values(by="Killed", ascending = False)

,Date,State,City,Address,Killed,Injured,Year,Code,Country,Latitude,Longitude,test
1247,01-Oct-17,Nevada,Las Vegas,3950 Las Vegas Blvd S,59,241,2017,NV,us,36.175,-115.136,False
1106,12-Jun-16,Florida,Orlando,1912 S Orange Avenue,50,53,2016,FL,us,28.5381,-81.3794,False
544,02-Dec-15,California,San Bernardino,1365 South Waterman Avenue,16,19,2015,CA,us,34.1083,-117.289,False
70,16-Sep-13,District of Columbia,Washington Navy Yard,1336 Isaac Hull Ave,11,3,2013,NaN,NaN,38.8746,-76.9945,True
592,01-Oct-15,Oregon,Roseburg,1140 Umpqua College Rd,10,9,2015,OR,us,43.2167,-123.341,False
1269,10-Sep-17,Texas,Plano,1712 West Spring Creek Parkway,9,1,2017,TX,us,33.0197,-96.6986,False
762,17-May-15,Texas,Waco,4671 S Jack Kultgen Fwy,9,18,2015,TX,us,31.5492,-97.1464,False
722,17-Jun-15,South Carolina,Charleston,110 Calhoun Street,9,0,2015,SC,us,32.7764,-79.9311,False
655,08-Aug-15,Texas,Houston,2211 Falling Oaks,8,0,2015,TX,us,29.7631,-95.3631,False
326,18-Sep-14,Florida,Bell,NW 30th Street and NW 39th Terrace,8,0,2014,FL,us,29.7553,-82.8628,False


In [26]:
df_merged_lon_lat[df_merged_lon_lat["City"] == "Pembroke Township"]

,Date,State,City,Address,Killed,Injured,Year,Code,Country,Latitude,Longitude,test
951,01-Oct-16,Illinois,Pembroke Township,17250E Road,4,0,2016,IL,NaN,45.81681,-77.11616,True


In [27]:
df_merged_lon_lat.loc[df_merged_lon_lat["City"] == "Pembroke Township", 'Latitude'] = "41.0667"
df_merged_lon_lat.loc[df_merged_lon_lat["City"] == "Pembroke Township", 'Longitude'] = "-87.6250348"


In [30]:
df_merged_lon_lat

,Date,State,City,Address,Killed,Injured,Year,Code,Country,Latitude,Longitude,test
0,31-Dec-13,New York,Brooklyn,60 Glenmore Ave,0,6,2013,NY,us,40.65,-73.95,False
1,28-Dec-13,Alabama,Montgomery,954 Highland Ave,3,5,2013,AL,us,32.3667,-86.3,False
2,26-Dec-13,Louisiana,Slidell,2144 First St,2,6,2013,LA,us,30.275,-89.7811,False
3,26-Dec-13,Louisiana,Lockport,313 Tenth St,3,3,2013,LA,us,29.6458,-90.5392,False
4,25-Dec-13,New Jersey,Irvington,Nye Avenue and 21st Street,3,2,2013,NJ,us,40.7322,-74.2353,False
5,25-Dec-13,New York,Medford,33A Cedarhurst Ave,1,3,2013,NY,us,40.8175,-73.0006,False
6,22-Dec-13,Michigan,Muskegon,2300 block of W McMillan Rd,1,3,2013,MI,us,43.2342,-86.2483,False
7,21-Dec-13,North Carolina,Shelby,Jose Street,0,4,2013,NC,us,35.2922,-81.5358,False
8,21-Dec-13,New Jersey,Trenton,Jersey Street and Home Avenue,1,4,2013,NJ,us,40.2169,-74.7433,False
9,15-Dec-13,Florida,Homestead,7th Avenue and 13th Terrace,1,3,2013,FL,us,25.4683,-80.4778,False


In [29]:
df_merged_lon_lat.to_csv("US_mass_shootings.csv")

In [34]:
df_merged_lon_lat_2017 = df_merged_lon_lat[df_merged_lon_lat["Year"] == 2017]

In [35]:
df_merged_lon_lat_2017

,Date,State,City,Address,Killed,Injured,Year,Code,Country,Latitude,Longitude,test
1242,09-Oct-17,Texas,Houston,3352 Old Spanish Trail,0,5,2017,TX,us,29.7631,-95.3631,False
1243,07-Oct-17,Arizona,Phoenix,87th Ave and Thomas Rd,0,4,2017,AZ,us,33.4483,-112.073,False
1244,06-Oct-17,Illinois,Chicago,8000 block of S Marshfield,0,5,2017,IL,us,41.85,-87.65,False
1245,05-Oct-17,Arizona,Casa Grande,400 block of W 13th Street,4,0,2017,AZ,us,32.8794,-111.757,False
1246,03-Oct-17,Florida,Miami,2051 NW 65th St,0,4,2017,FL,us,25.7739,-80.1939,False
1247,01-Oct-17,Nevada,Las Vegas,3950 Las Vegas Blvd S,59,241,2017,NV,us,36.175,-115.136,False
1248,01-Oct-17,Kansas,Lawrence,1000 block of Massachusetts St,3,2,2017,KS,us,38.9717,-95.235,False
1249,30-Sep-17,Tennessee,Memphis,800 block of Kerr Avenue,1,3,2017,TN,us,35.1494,-90.0489,False
1250,28-Sep-17,Louisiana,New Orleans,2001 N Rocheblave St,1,3,2017,LA,us,29.9544,-90.075,False
1251,26-Sep-17,Tennessee,Memphis,2531 Chelsea Ave,0,4,2017,TN,us,35.1494,-90.0489,False


In [36]:
df_merged_lon_lat_2017.to_csv("US_mass_shootings_2017.csv")

In [48]:
df_merged_lon_lat_2017_grouped = df_merged_lon_lat_2017.groupby("Date")["Killed", "Injured"].sum().reset_index()

In [49]:
df_merged_lon_lat_2017_grouped.dtypes

Date       object
Killed      int64
Injured     int64
dtype: object

In [52]:
df_merged_lon_lat_2017_grouped["Date"] = pd.to_datetime(df_merged_lon_lat_2017_grouped["Date"], format="%d-%b-%y")

In [53]:
df_merged_lon_lat_2017_grouped.head()

,Date,Killed,Injured
0,2017-04-01,3,12
1,2017-08-01,0,17
2,2017-01-01,1,15
3,2017-07-01,2,35
4,2017-06-01,3,1


In [55]:
df_merged_lon_lat_2017_grouped = df_merged_lon_lat_2017_grouped.sort_values(by = "Date", ascending = True)

In [56]:
df_merged_lon_lat_2017_grouped.to_csv("US_mass_shootings_2017_by_date.csv")

In [58]:
df_merged_lon_lat_2017_grouped

,Date,Killed,Injured
2,2017-01-01,1,15
15,2017-01-03,0,4
22,2017-01-04,3,1
35,2017-01-06,7,8
42,2017-01-07,1,3
63,2017-01-11,2,6
67,2017-01-12,5,4
77,2017-01-15,4,5
83,2017-01-16,0,12
99,2017-01-20,1,8
